In [1]:
#1. Generation dataset
#2. Training classifier and save it
#3. Detect the face and named it if it is already stored in our dataset

In [2]:
!git clone https://github.com/DucMinhNgo/face-time-attendance-system.git

Cloning into 'face-time-attendance-system'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 0), reused 8 (delta 0), pack-reused 0
Receiving objects: 100% (8/8), 143.65 KiB | 1.17 MiB/s, done.


In [3]:
%cd /content/face-time-attendance-system

[Errno 2] No such file or directory: '/content/face-time-attendance-system'
/Users/dustin/Desktop/Tech/face-time-attendance-system


In [4]:
# Generation
!pip install opencv-python

In [5]:
import cv2

In [9]:
def generate_dataset():
  face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
  def face_cropped(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    # Scaling factor= 1.3
    # Minimum neighbor = 5

    if faces is ():
      return None

    for (x, y, w, h) in faces:
      cropped_face = img[y:y+h, x:x+w]

    return cropped_face

  cap = cv2.VideoCapture(0)
  id=1
  img_id=0

  while True:
    ret, frame = cap.read()
    if face_cropped(frame) is not None:
      img_id+=1
      face = cv2.resize(face_cropped(frame), (200, 200))
      face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
      file_name_path = "data/user." + str(id) + "." + str(img_id) + ".jpg"
      cv2.imwrite(file_name_path, face)
      cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
      # (50, 50) is the origin point from where text is to be written
      # font scale= 1
      # thickness = 2
      cv2.imshow("Cropped face", face)
      if cv2.waitKey(1) == 13 or int(img_id) == 200:
        break
  cap.release()
  cv2.destroyAllWindows()
  print("Collecting samples is completed")
generate_dataset()


<>:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
/var/folders/r5/5v_j803101v_nmxqj00wtl100000gn/T/ipykernel_38899/3356530203.py:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():
/var/folders/r5/5v_j803101v_nmxqj00wtl100000gn/T/ipykernel_38899/3356530203.py:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():
/var/folders/r5/5v_j803101v_nmxqj00wtl100000gn/T/ipykernel_38899/3356530203.py:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():
/var/folders/r5/5v_j803101v_nmxqj00wtl100000gn/T/ipykernel_38899/3356530203.py:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


KeyboardInterrupt: 

In [10]:
cv2.destroyAllWindows()

In [11]:
!pip install numpy
!pip install Pillow

In [4]:
import numpy as np
from PIL import Image
import os
import cv2

In [14]:
def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])

        faces.append(imageNp)
        ids.append(id)
    
    ids = np.array(ids)

    # Train the classifier and save
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")

train_classifier("data")

# Detect the face and named it if it is already stored in our dataset

In [5]:
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbors)

    coords = []

    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
        id, pred = clf.predict(gray_image[y:y+h, x:x+w])
        confidence = int(100*(1-pred/300))

        if confidence > 77:
            if id == 1:
                cv2.putText(img, "Dustin", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "UNKNOWN", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        
        coords = [x, y, w, h]
    return coords

def recognize(img, clf, faceCascade):
    coords = draw_boundary(img, faceCascade, 1.1, 10, (255, 255, 255), "Face", clf)
    return img


faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

video_capture = cv2.VideoCapture(0)

while True:
    ret, img = video_capture.read()
    img = recognize(img, clf, faceCascade)
    cv2.imshow("Face Detection", img)

    if cv2.waitKey(1) == 13:
        break

video_capture.release()
cv2.destroyAllWindows()